### Build Docker Image that contains Resnet 152 model and Flask web application
Make sure you are able to run Docker without sudo.  
Make sure you are have logged in using docker login. 


In [1]:
import os
from os import path
import json

In [2]:
!mkdir flaskwebapp
!mkdir flaskwebapp/nginx
!mkdir flaskwebapp/etc

Pull in Resnet 152 model

In [3]:
!wget "https://migonzastorage.blob.core.windows.net/deep-learning/models/cntk/imagenet/ResNet_152.model"

--2017-04-16 13:15:50--  https://migonzastorage.blob.core.windows.net/deep-learning/models/cntk/imagenet/ResNet_152.model
Resolving migonzastorage.blob.core.windows.net (migonzastorage.blob.core.windows.net)... 191.235.192.206
Connecting to migonzastorage.blob.core.windows.net (migonzastorage.blob.core.windows.net)|191.235.192.206|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 242117624 (231M) [application/octet-stream]
Saving to: ‘ResNet_152.model’

100%[======================================>] 242,117,624 12.4MB/s   in 20s    

2017-04-16 13:16:11 (11.3 MB/s) - ‘ResNet_152.model’ saved [242117624/242117624]



Pull in class labels

In [4]:
!wget "https://ikcompuvision.blob.core.windows.net/acs/synset.txt"

--2017-04-16 13:16:11--  https://ikcompuvision.blob.core.windows.net/acs/synset.txt
Resolving ikcompuvision.blob.core.windows.net (ikcompuvision.blob.core.windows.net)... 52.239.158.68
Connecting to ikcompuvision.blob.core.windows.net (ikcompuvision.blob.core.windows.net)|52.239.158.68|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31675 (31K) [text/plain]
Saving to: ‘synset.txt’

100%[======================================>] 31,675      --.-K/s   in 0s      

2017-04-16 13:16:11 (196 MB/s) - ‘synset.txt’ saved [31675/31675]



In [5]:
!cp ResNet_152.model flaskwebapp
!cp synset.txt flaskwebapp
!ls flaskwebapp

etc  nginx  ResNet_152.model  synset.txt


Below is the driver for our model. The methods in this module will be called by our webapp.

In [6]:
%%writefile flaskwebapp/driver.py
import numpy as np
import logging, sys, json
import timeit as t
import base64
from cntk import load_model, combine
from PIL import Image, ImageOps
from io import BytesIO


logger = logging.getLogger("cntk_svc_logger")
ch = logging.StreamHandler(sys.stdout)
logger.addHandler(ch)

trainedModel = None
mem_after_init = None
labelLookup = None
topResult = 3


def init():
    """ Initialise ResNet 152 model
    """
    global trainedModel, labelLookup, mem_after_init

    start = t.default_timer()
    
    # Load the model and labels from disk
    with open('synset.txt', 'r') as f:
        labelLookup = [l.rstrip() for l in f]
        
    # Load model and load the model from brainscript (3rd index)
    trainedModel = load_model('ResNet_152.model')
    trainedModel = combine([trainedModel.outputs[3].owner])
    end = t.default_timer()

    loadTimeMsg = "Model loading time: {0} ms".format(round((end-start)*1000, 2))
    logger.info(loadTimeMsg)

    
def run(inputString):
    """ Classify the input using the loaded model
    """
    start = t.default_timer()

    images=json.loads(inputString)
    result = []
    totalPreprocessTime = 0
    totalEvalTime = 0
    totalResultPrepTime = 0

    for base64ImgString in images:
        if base64ImgString.startswith('b\''):
            base64ImgString = base64ImgString[2:-1]
        base64Img = base64ImgString.encode('utf-8')

        # Preprocess the input data 
        startPreprocess = t.default_timer()
        decoded_img = base64.b64decode(base64Img)
        img_buffer = BytesIO(decoded_img)
        
        # Load image with PIL (RGB)
        pil_img = Image.open(img_buffer).convert('RGB')
        pil_img = ImageOps.fit(pil_img, (224, 224), Image.ANTIALIAS)
        rgb_image = np.array(pil_img, dtype=np.float32)
        
        # Resnet trained with BGR
        bgr_image = rgb_image[..., [2, 1, 0]]
        imageData = np.ascontiguousarray(np.rollaxis(bgr_image, 2))

        endPreprocess = t.default_timer()
        totalPreprocessTime += endPreprocess - startPreprocess

        # Evaluate the model using the input data
        startEval = t.default_timer()
        imgPredictions = np.squeeze(trainedModel.eval({trainedModel.arguments[0]:[imageData]}))
        endEval = t.default_timer()
        totalEvalTime += endEval - startEval

        # Only return top 3 predictions
        startResultPrep = t.default_timer()
        resultIndices = (-np.array(imgPredictions)).argsort()[:topResult]
        imgTopPredictions = []
        for i in range(topResult):
            imgTopPredictions.append((labelLookup[resultIndices[i]], imgPredictions[resultIndices[i]] * 100))
        endResultPrep = t.default_timer()
        result.append(imgTopPredictions)

        totalResultPrepTime += endResultPrep - startResultPrep

    end = t.default_timer()

    logger.info("Predictions: {0}".format(result))
    logger.info("Predictions took {0} ms".format(round((end-start)*1000, 2)))
    logger.info("Time distribution: preprocess={0} ms, eval={1} ms, resultPrep = {2} ms".format(round(totalPreprocessTime * 1000, 2), round(totalEvalTime * 1000, 2), round(totalResultPrepTime * 1000, 2)))

    actualWorkTime = round((totalPreprocessTime + totalEvalTime + totalResultPrepTime)*1000, 2)
    return (result, 'Computed in {0} ms'.format(actualWorkTime))

Writing flaskwebapp/driver.py


Below is the module for the Flask web application.

In [7]:
%%writefile flaskwebapp/app.py
from flask import Flask, request
import cntk
from driver import *
import time

app = Flask(__name__)


@app.route('/score', methods = ['POST'])
def scoreRRS():
    """ Endpoint for scoring
    """
    if request.headers['Content-Type'] != 'application/json':
        return Response(json.dumps({}), status= 415, mimetype ='application/json')
    input = request.json['input']
    start = time.time()
    response = run(input)
    end = time.time() - start
    dict = {}
    dict['result'] = response
    return json.dumps(dict)


@app.route("/")
def healthy():
    return "Healthy"


# CNTK Version
@app.route('/version', methods = ['GET'])
def version_request():
    return cntk.__version__


if __name__ == "__main__":
    app.run(host='0.0.0.0') # Ignore, Development server

Writing flaskwebapp/app.py


In [8]:
%%writefile flaskwebapp/wsgi.py
import sys
sys.path.append('/code/') # FIXME: This is horrible
from app import app as application
from driver import *

def create():
    print("Initialising")
    init()
    application.run(host='127.0.0.1', port=5000)

Writing flaskwebapp/wsgi.py


In [9]:
%%writefile flaskwebapp/requirements.txt
pillow
click==6.7
configparser==3.5.0
Flask==0.11.1
gunicorn==19.6.0
json-logging-py==0.2
MarkupSafe==1.0
olefile==0.44
requests==2.12.3

Writing flaskwebapp/requirements.txt


The configuration for the Nginx. Note that it creates a proxy between ports **88** and **5000**.

In [10]:
%%writefile flaskwebapp/nginx/app
server {
    listen 88;
    server_name _;
 
    location / {
    include proxy_params;
    proxy_pass http://127.0.0.1:5000;
    proxy_connect_timeout 5000s;
    proxy_read_timeout 5000s;
  }
}

Writing flaskwebapp/nginx/app


In [11]:
image_name = "masalvar/cntkresnet"
application_path = 'flaskwebapp'
docker_file_location = path.join(application_path, 'dockerfile')

In [12]:
%%writefile flaskwebapp/gunicorn_logging.conf

[loggers]
keys=root, gunicorn.error

[handlers]
keys=console

[formatters]
keys=json

[logger_root]
level=INFO
handlers=console

[logger_gunicorn.error]
level=ERROR
handlers=console
propagate=0
qualname=gunicorn.error

[handler_console]
class=StreamHandler
formatter=json
args=(sys.stdout, )

[formatter_json]
class=jsonlogging.JSONFormatter

Writing flaskwebapp/gunicorn_logging.conf


In [13]:
%%writefile flaskwebapp/kill_supervisor.py
import sys
import os
import signal


def write_stdout(s):
    sys.stdout.write(s)
    sys.stdout.flush()

# this function is modified from the code and knowledge found here: http://supervisord.org/events.html#example-event-listener-implementation
def main():
    while 1:
        write_stdout('READY\n')
        # wait for the event on stdin that supervisord will send
        line = sys.stdin.readline()
        write_stdout('Killing supervisor with this event: ' + line);
        try:
            # supervisord writes its pid to its file from which we read it here, see supervisord.conf
            pidfile = open('/tmp/supervisord.pid','r')
            pid = int(pidfile.readline());
            os.kill(pid, signal.SIGQUIT)
        except Exception as e:
            write_stdout('Could not kill supervisor: ' + e.strerror + '\n')
            write_stdout('RESULT 2\nOK')

main()


Writing flaskwebapp/kill_supervisor.py


In [14]:
%%writefile flaskwebapp/etc/supervisord.conf 
[supervisord]
logfile=/tmp/supervisord.log ; (main log file;default $CWD/supervisord.log)
logfile_maxbytes=50MB        ; (max main logfile bytes b4 rotation;default 50MB)
logfile_backups=10           ; (num of main logfile rotation backups;default 10)
loglevel=info                ; (log level;default info; others: debug,warn,trace)
pidfile=/tmp/supervisord.pid ; (supervisord pidfile;default supervisord.pid)
nodaemon=true               ; (start in foreground if true;default false)
minfds=1024                  ; (min. avail startup file descriptors;default 1024)
minprocs=200                 ; (min. avail process descriptors;default 200)

[program:gunicorn]
command=bash -c "gunicorn --workers 1 -m 007 --timeout 100000 --capture-output --error-logfile - --log-level debug --log-config gunicorn_logging.conf \"wsgi:create()\""
directory=/code
redirect_stderr=true
stdout_logfile =/dev/stdout
stdout_logfile_maxbytes=0
startretries=2
startsecs=20

[program:nginx]
command=/usr/sbin/nginx -g "daemon off;"
startretries=2
startsecs=5
priority=3

[eventlistener:program_exit]
command=python kill_supervisor.py
directory=/code
events=PROCESS_STATE_FATAL
priority=2

Writing flaskwebapp/etc/supervisord.conf


We create a custom image based on Ubuntu 16.04 and install all the necessary dependencies. This is in order to try and keep the size of the image as small as possible.

In [15]:
%%writefile flaskwebapp/dockerfile

FROM ubuntu:16.04
MAINTAINER Mathew Salvaris <mathew.salvaris@microsoft.com>

RUN mkdir /code
WORKDIR /code
ADD . /code/
ADD etc /etc

RUN apt-get update && apt-get install -y --no-install-recommends \
        openmpi-bin \
        python \ 
        python-dev \ 
        python-setuptools \
        python-pip \
        supervisor \
        nginx && \
    rm /etc/nginx/sites-enabled/default && \
    cp /code/nginx/app /etc/nginx/sites-available/ && \
    ln -s /etc/nginx/sites-available/app /etc/nginx/sites-enabled/ && \
    pip install https://cntk.ai/PythonWheel/CPU-Only/cntk-2.0rc1-cp27-cp27mu-linux_x86_64.whl && \
    pip install -r /code/requirements.txt

EXPOSE 88
CMD ["supervisord", "-c", "/etc/supervisord.conf"]

Writing flaskwebapp/dockerfile


In [16]:
!docker build -t $image_name -f $docker_file_location $application_path --no-cache

Sending build context to Docker daemon 242.2 MB
Step 1 : FROM ubuntu:16.04
 ---> 0ef2e08ed3fa
Step 2 : MAINTAINER Mathew Salvaris <mathew.salvaris@microsoft.com>
 ---> Running in 5eada5173ba6
 ---> 0ab56bc88dd3
Removing intermediate container 5eada5173ba6
Step 3 : RUN mkdir /code
 ---> Running in 7e87123877d2
 ---> 2a64272fc540
Removing intermediate container 7e87123877d2
Step 4 : WORKDIR /code
 ---> Running in 5bbe5d1cb8a1
 ---> 75c7f63947ee
Removing intermediate container 5bbe5d1cb8a1
Step 5 : ADD . /code/
 ---> 0d98a7c447ed
Removing intermediate container 0b45d3cf846b
Step 6 : ADD etc /etc
 ---> 95f0a053a550
Removing intermediate container 76ed7d88877a
Step 7 : RUN apt-get update && apt-get install -y --no-install-recommends         openmpi-bin         python         python-dev         python-setuptools         python-pip         supervisor         nginx &&     rm /etc/nginx/sites-enabled/default &&     cp /code/nginx/app /etc/nginx/sites-available/ &&     ln -s /etc/nginx/sites-ava

In [17]:
!docker push $image_name

The push refers to a repository [docker.io/masalvar/cntkresnet]









latest: digest: sha256:84bfc0636227b856d59c9c328bdfd97d68f3210565f0ce620195d6c85a4bb7b4 size: 2196


In [19]:
print('Docker image name {}'.format(image_name)) 

Docker image name masalvar/cntkresnet


### Test locally
Go to the [Test Locally notebook](TestLocally.ipynb) to test your Docker image